In [39]:
from stravalib import Client
import os 
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd 

load_dotenv() 

def get_strava_authorization_code():

    client = Client()
    url = client.authorization_url(
        client_id=os.environ["CLIENT_ID"] ,
        redirect_uri="http://127.0.0.1:5000/authorization",
    ) 
    print(url)

def get_strava_token(code):

    client = Client()
    token_response = client.exchange_code_for_token(
        client_id=os.environ["CLIENT_ID"], client_secret=os.environ["CLIENT_SECRET"], code=code
    )

    return token_response

def strava_login(token_response):

    client = Client(
        access_token=token_response["access_token"],
    )

    athlete = client.get_athlete()
    print(f"Hi, {athlete.firstname} Welcome to stravalib!")

    return client 

def strava_connection(): 

    get_strava_authorization_code()
    code = input("Insert Autorization Code: ")
    token_response = get_strava_token(code)
    client = strava_login(token_response)

    return client

In [3]:
client = strava_connection()

https://www.strava.com/oauth/authorize?client_id=131915&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauthorization&approval_prompt=auto&scope=read%2Cactivity%3Aread&response_type=code


No rates present in response headers


Hi, Bruno Welcome to stravalib!


In [50]:
import os 
data_path = "data"
already_read_files = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and f.endswith('csv')]
already_read_files

['PO 12k__20250304.csv', 'San Cristóbal Virgen__20250223.csv']

In [ ]:
activities = client.get_activities(limit=20)

for a in activities: 
    id = a.id
    activity_name = a.name
    activity_type = a.type
    start_date_date = a.start_date
    start_date_str = start_date_date.strftime("%Y%m%d")
    distance = f"{float(a.distance)/1000:.2f}K"

    csv_name = f"{activity_name}__{start_date_str}.csv" 

    if (activity_type not in ["WeightTraining", "Workout", "VirtualRide", "Treadmill"]) and (csv_name not in already_read_files): 

        activity = client.get_activity_streams(id, types = ["latlng", "altitude"])

        try:
            latlng = activity["latlng"].data
            altitude = activity["altitude"].data
            data_ = pd.DataFrame([*latlng], columns=['lat','long'])
            data_['altitude'] = altitude
            data_["distance"] = distance
            data_['activity_id'] = id 
            data_['activity_name'] = activity_name        
            data_['activity_type'] = activity_type    
            data_.to_csv(f'data//{csv_name}', index = False)
            print(f"Activity {activity_name} successfully saved")
        except KeyError:
            print("Activity doesn't have gps data")

Activity doesn't have gps data
Activity doesn't have gps data
Activity doesn't have gps data
Activity doesn't have gps data
Activity doesn't have gps data
Activity doesn't have gps data
Activity doesn't have gps data


Short term API rate limit exceeded
